In [27]:
import pandas as pd
import numpy as np
import re
import operator

In [28]:
worried_data_path = '../Data/Raw/worried_data.csv'

In [29]:
worried_data = pd.read_csv(worried_data_path)
worried_data.rename(columns = {'2011 [YR2011]': 2011, '2014 [YR2014]': 2014, '2017 [YR2017]': 2017, '2021 [YR2021]': 2021, '2022 [YR2022]': 2022}, inplace = True)
# drop all rows where no country name or series code is provided
# it will remove all data description rows generated while exporting from Findex Database
worried_data.dropna(subset = ['Country Name', 'Series Code'], inplace = True)

In [30]:
series_codes = worried_data["Series Code"].unique()
series_desc = {}
for i in range(len(series_codes)):
    code = series_codes[i]
    series_name = worried_data[worried_data["Series Code"] ==  code]["Series Name"].unique()
    if series_name.shape[0] == 1:
        series_desc[code] = series_name.item()
    elif series_name.shape[0] == 0:
        print("No series name for", code)
    elif series_name.shape[0] > 1:
        print("Something went wrong")

In [31]:
# For the purpose of documenting the data used in the README
with open("../Data/Processed/worried_desc.txt", "w") as worried_data_desc:
    worried_data_desc.write("\n".join([" ".join(["|", key, "|", series_desc[key], "|"]) for key in series_desc.keys()]))

In [32]:
# The data is sorted according to the specific series, so the relevant series are grouped together for the purpose of clarity
for key in series_desc.keys():
    m = re.search('(.*):(.*?)([(,].*)', series_desc[key])
    series_name = " ".join([m.groups()[0].strip(), m.groups()[2].strip(" ,"), m.groups()[1].strip()])
    series_desc[key] = series_name
series_desc = dict(sorted(series_desc.items(), key = operator.itemgetter(1)))

In [33]:
# Values exist in 2021 and 2022 columns (exclusively), all values from 2011, 2014 and 2017 are Nan
# change strings to numeric values, use NaN wherever data not available
value_columns = [2011, 2014, 2017, 2021, 2022]
worried_data[value_columns] = worried_data[value_columns].apply(pd.to_numeric, errors = 'coerce')
worried_data.drop(columns = ['Series Name', 2011, 2014, 2017], inplace = True)
# store values in percentage columns
worried_data['percentage'] = worried_data[2021].fillna(worried_data[2022])
worried_data.drop(columns = [2021, 2022], inplace = True)

In [34]:
countries = np.array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cyprus', 'Czechia',
       'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt, Arab Rep.', 'El Salvador', 'Estonia', 'Eswatini',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Gambia, The', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Haiti',
       'Honduras', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Israel',
       'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
       'Korea, Rep.', 'Kosovo', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR',
       'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania',
       'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives',
       'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Moldova',
       'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar',
       'Namibia', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua',
       'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Oman',
       'Pakistan', 'Panama', 'Paraguay', 'Peru', 'Philippines', 'Poland',
       'Portugal', 'Puerto Rico', 'Qatar', 'Romania',
       'Russian Federation', 'Rwanda', 'Saudi Arabia', 'Senegal',
       'Serbia', 'Sierra Leone', 'Singapore', 'Slovak Republic',
       'Slovenia', 'Somalia', 'South Africa', 'South Sudan', 'Spain',
       'Sri Lanka', 'Sudan', 'Sweden', 'Switzerland',
       'Syrian Arab Republic', 'Taiwan, China', 'Tajikistan', 'Tanzania',
       'Thailand', 'Togo', 'Trinidad and Tobago', 'Tunisia', 'Turkiye',
       'Turkmenistan', 'Uganda', 'Ukraine', 'United Arab Emirates',
       'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan',
       'Venezuela, RB', 'Vietnam', 'West Bank and Gaza', 'Yemen, Rep.',
       'Zambia', 'Zimbabwe'])
not_countries = ['Arab World', 'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)', 'High income',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle income', 'North America', 'OECD members', 'South Asia',
       'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
       'Upper middle income', 'World']

In the Findex database values are provided in columns corresponding to years representing the survey waves. In 2024 the data about what population is worried about is available for the last wave (years 2021 and 2022). The data is pivoted, it is indexed by the name of the country, and columns represent series. Values are concatenated from 2021 and 2022.

In [64]:
worried_data_pivoted = []
for country in countries:
    filtered_data = worried_data[(worried_data["Country Name"] == country)].copy()
    # ordered_filtered_data = filtered_data.set_index("Series Code").reindex(series_codes)
    worried_data_pivoted.append(filtered_data.pivot(index = ['Country Name', 'Country Code'], columns = 'Series Code', values = 'percentage').reset_index(level = 1))

for country in not_countries:
#for country in ['Azerbaijan']:
    filtered_data = worried_data[(worried_data["Country Name"] == country)].copy()
    # ordered_filtered_data = filtered_data.set_index("Series Code").reindex(series_codes)
    worried_data_pivoted.append(filtered_data.pivot(index = ['Country Name', 'Country Code'], columns = 'Series Code', values = 'percentage').reset_index(level = 1))

worried_data_pivoted = pd.concat(worried_data_pivoted)
# order columns
# it doesn't include "Country Code" column, so it needs to be added
ordered_worried_data_pivoted = worried_data_pivoted[series_desc.keys()].copy()

# some countries have no data
# drop countries with all missing values
ordered_worried_data_pivoted.dropna(axis = 'index', how = 'all', inplace = True)
# some of the columns/series have less than 50% of data present (rural/urban series)
# drop columns with 50% or more missing values
ordered_worried_data_pivoted.dropna(axis = 'columns', thresh = 0.5*len(ordered_worried_data_pivoted.columns), inplace = True)

ordered_worried_data_pivoted["Country Code"] = worried_data_pivoted["Country Code"].copy()

In [66]:
countries_data = ordered_worried_data_pivoted[ordered_worried_data_pivoted.index.isin(countries)].copy()
countries_data.head()

Series Code,fin44b3.d,fin44b2.d,fin44b1.d,fin44b3.d.1,fin44b2.d.1,fin44b1.d.1,fin44b3.d.12,fin44b2.d.12,fin44b1.d.12,fin44b3.d.7,...,fin44a3.d.5,fin44a2.d.5,fin44a1.d.5,fin44a3.d.6,fin44a2.d.6,fin44a1.d.6,fin44a3.d.3,fin44a2.d.3,fin44a1.d.3,Country Code
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.69,16.79,82.52,0.95,14.29,84.76,0.37,15.89,83.74,0.00,...,5.72,33.85,60.36,9.05,40.60,50.35,8.51,38.66,52.68,AFG
Albania,9.24,19.47,70.74,8.85,16.51,74.28,8.55,21.73,69.32,6.14,...,16.77,18.31,64.38,23.94,31.07,38.83,45.85,28.35,17.20,ALB
Algeria,19.77,45.99,27.92,15.84,45.99,30.78,17.14,47.73,29.62,11.79,...,34.14,30.95,26.65,39.67,33.07,14.54,45.24,23.90,14.21,DZA
Argentina,26.15,27.10,45.25,21.23,26.50,51.72,23.11,29.53,45.96,17.16,...,28.66,23.38,45.48,26.96,42.04,30.88,20.58,52.55,26.87,ARG
Armenia,24.06,34.19,40.26,19.42,33.33,45.86,26.35,35.13,37.22,14.88,...,34.54,34.30,29.68,40.27,28.29,28.60,61.04,29.98,7.00,ARM


There are 139 countries each with 132 long feature vector. THere's additional column with country code which is necesary for future use with geopandas and naturalearth datasets.

In [67]:
# The number and percentage of missing values
missing = countries_data.isnull().sum().sum()
print("There are", missing, "missing values.")
perc_missing = np.round(missing/(countries_data.values.shape[0] * countries_data.values.shape[1]) * 100, 2)
print("There are", perc_missing, "% of values missing.")

There are 24 missing values.
There are 0.13 % of values missing.


In [68]:
# Missing values imputation
# The world aggregate for each series is used for missing values
countries_data.fillna(ordered_worried_data_pivoted.loc["World"], inplace = True)

In [69]:
missing = countries_data.isnull().sum().sum()
print("There are", missing, "missing values.")
perc_missing = np.round(missing/(countries_data.values.shape[0] * countries_data.values.shape[1]) * 100, 2)
print("There are", perc_missing, "% of values missing.")

There are 0 missing values.
There are 0.0 % of values missing.


In [70]:
countries_data

Series Code,fin44b3.d,fin44b2.d,fin44b1.d,fin44b3.d.1,fin44b2.d.1,fin44b1.d.1,fin44b3.d.12,fin44b2.d.12,fin44b1.d.12,fin44b3.d.7,...,fin44a3.d.5,fin44a2.d.5,fin44a1.d.5,fin44a3.d.6,fin44a2.d.6,fin44a1.d.6,fin44a3.d.3,fin44a2.d.3,fin44a1.d.3,Country Code
Country Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.69,16.79,82.52,0.95,14.29,84.76,0.37,15.89,83.74,0.00,...,5.72,33.85,60.36,9.05,40.60,50.35,8.51,38.66,52.68,AFG
Albania,9.24,19.47,70.74,8.85,16.51,74.28,8.55,21.73,69.32,6.14,...,16.77,18.31,64.38,23.94,31.07,38.83,45.85,28.35,17.20,ALB
Algeria,19.77,45.99,27.92,15.84,45.99,30.78,17.14,47.73,29.62,11.79,...,34.14,30.95,26.65,39.67,33.07,14.54,45.24,23.90,14.21,DZA
Argentina,26.15,27.10,45.25,21.23,26.50,51.72,23.11,29.53,45.96,17.16,...,28.66,23.38,45.48,26.96,42.04,30.88,20.58,52.55,26.87,ARG
Armenia,24.06,34.19,40.26,19.42,33.33,45.86,26.35,35.13,37.22,14.88,...,34.54,34.30,29.68,40.27,28.29,28.60,61.04,29.98,7.00,ARM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,17.24,42.04,40.34,15.78,39.60,44.10,16.04,43.38,40.49,11.00,...,21.68,39.02,39.31,37.77,41.97,19.18,58.17,26.65,10.86,VNM
West Bank and Gaza,19.07,27.91,52.03,21.25,27.25,50.32,19.00,27.14,53.02,9.56,...,27.77,25.88,45.78,32.79,31.56,32.62,37.38,32.71,25.32,PSE
"Yemen, Rep.",12.15,28.85,57.41,14.60,26.51,57.04,7.66,24.80,66.64,5.81,...,29.05,24.70,40.17,37.18,27.94,25.55,40.44,20.23,25.42,YEM


In [71]:
countries_data.to_csv("../Data/Processed/worried_data_processed.csv")